In [19]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import WebDriverException, NoSuchWindowException


Notebook này được sử dụng để thu thập (crawl) thông tin về các bộ phim phổ biến từ trang IMDB, cụ thể tại đường dẫn: https://www.imdb.com/search/title/?title_type=feature

**Lưu ý**: Cấu trúc HTML cũng như giao diện của website IMDB có thể thay đổi theo thời gian. Do đó, các selector và logic crawl trong notebook có thể không còn hoạt động chính xác trong tương lai và cần được cập nhật định kỳ để đảm bảo quá trình thu thập dữ liệu vẫn thực thi đúng


## 1. Setup driver


khởi tạo trình duyệt Edge để phục vụ cho quá trình crawl dữ liệu trên IMDb.\
Hàm setup tạo WebDriver theo cấu hình mặc định của Edge và có thể quan sát trực tiếp toàn bộ quá trình crawl — phù hợp khi debug hoặc kiểm thử thủ công.

In [20]:
def setup(driver_path):
    try:
        cService =webdriver.EdgeService(executable_path = driver_path)
        
        driver= webdriver.Edge(service=cService)
        return driver
    except:
        print('Something went wrong!!')

## 2. Crawl phim

Sử dụng hàm `get_movie_details` để lấy các thông tin không xuất hiện trong danh sách ngoài, bao gồm:
- Genres 
- Budget

In [21]:
def get_movie_details(driver, url):
    main_tab = driver.current_window_handle
    genres = None
    budget = None
    try:
        # mở tab mới
        driver.execute_script("window.open('');")
        new_tab = [t for t in driver.window_handles if t != main_tab][0]
        driver.switch_to.window(new_tab)
        # load trang phim
        driver.get(url)
        # Chờ page load 
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "h1")) )
        try:
            driver.title   # nếu Edge bị tắt → lỗi tại đây
        except:
            print(f"⛔ Trình duyệt Edge đã bị tắt khi đang xử lý phim {global_idx}. Dừng crawl!")
            return data
        # ====== LẤY GENRES ======
        try:
            # scroll đến storyline
            storyline = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//span[text()="Storyline"]')))
            driver.execute_script("arguments[0].scrollIntoView(true);", storyline)

            genre_block = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//li[@data-testid="storyline-genres"]')))
            items = genre_block.find_elements(By.XPATH, './/ul/li')
            genres = ", ".join([i.text for i in items if i.text]) or None
        except:
            genres = None
         # ====== LẤY BUDGET ======
        try:
            li = WebDriverWait(driver, 4).until(EC.presence_of_element_located
                                                ((By.XPATH, '//div[@data-testid="title-boxoffice-section"]//li[@data-testid="title-boxoffice-budget"]')) )
            budget_span = li.find_element(By.XPATH, './/span[contains(@class,"ipc-metadata-list-item__list-content-item")]')
            budget = budget_span.text.strip()
        except:
            budget = None

    finally:
        # đóng tab mới
        try:
            driver.close()
        except:
            pass
        driver.switch_to.window(main_tab)
    return genres, budget

Hàm này dùng để:
```    
1. Mở link url , đợi sleep 3s để trang load hết
2. tạo list data để lưu kết quả crawl 10000 phim, với mỗi phần từ là 1 dictionary lưu các thông tin phim
3. Vòng lặp duyệt sô trang (`num_page`)
        nếu page == end:( duyệt tới trang mong muốn)
            break
        nếu page < star: ( đang duyệt tới trang chưa mong muốn)
            lưu container(để tránh miss), kiểm tra số lượng container 
            đợi trang load hết, kéo xuống gần cuối trang chỗ button sleep 1s đợi load button
            bấm vào button
            sleep 3s để 50 phim tiếp theo load
        nếu page nằm trong khoảng [start,end]: 
            lấy container, kiểm tra số lượng container
            lấy link phim
            vòng lặp duyệt 50 phim hiện tại:
                lấy rank, title, release_year, run_time, mpa, metascore, vote_count, rating
                mở tab mới để truy cập chi tiết vào link phim: kéo xuống tới Storyline, lấy genre và budget, đóng tab, trở về
                kéo xuống gần cuối trang chỗ button sleep 1s đợi load button
                bấm vào button
                sleep 3s để 50 phim tiếp theo load
```               

In [22]:
def crawl_imdb_popular_movies(driver_path, num_pages=201,start =0, end =1):
    url = 'https://www.imdb.com/search/title/?title_type=feature'
    driver = setup(driver_path)
    
    if driver is None:
        return []
    driver.get(url)
    time.sleep(3)

    row_template = {
        'rank': np.nan,
        'title': np.nan,
        'genres': np.nan,
        'budget': np.nan,
        'release_year': np.nan,
        'run_time': np.nan,
        'mpa': np.nan,
        'metascore': np.nan,
        'vote_count': np.nan,
        'rating': np.nan
    }
    
    data = []
    load_more_xpath = '//button[contains(@class, "ipc-see-more__button")]'
    try:  
        for page in range(num_pages):
            if driver is None or driver.session_id is None:
                print("⛔ Trình duyệt Edge đã bị đóng! Dừng crawl.")
                return data
            print(f"\n=== PAGE {page+1} ===")
            if page == end: 
                print('đã crawl xong')
                break
            elif page < start:
                containers = driver.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')
                expected_total = 50 * (page+1)
                if len(containers) < expected_total:
                    print(f"⚠ IMDb chưa load đủ. Đang thấy: {len(containers)}, cần: {expected_total}")                  
                    # chờ thêm để load 
                    try:
                        WebDriverWait(driver, 10).until(lambda d: len(
                                d.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')) >= expected_total)
                    except:
                        pass                
                    # kiểm tra lần cuối
                    containers = driver.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')
                    if len(containers) < expected_total:
                        print(f"❌ IMDb KHÔNG LOAD ĐỦ SAU KHI CHỜ")
                        print(f"Đang có {len(containers)}, cần {expected_total}")
                        break
                try:
                    load_more_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, load_more_xpath)))
                    driver.execute_script("arguments[0].scrollIntoView(true);", load_more_btn)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", load_more_btn)
                    time.sleep(3)
                except:
                    print("Không còn phim để load.")
                    break
            else:
                # Lấy tất cả phim đã load
                containers = driver.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')
                expected_total = 50 * (page+1)
    
                if len(containers) < expected_total:
                    print(f"⚠ IMDb chưa load đủ. Đang thấy: {len(containers)}, cần: {expected_total}")            
                    try:
                        WebDriverWait(driver, 10).until(
                            lambda d: len(
                                d.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')
                            ) >= expected_total
                        )
                    except:
                        pass
                
                    # kiểm tra lần cuối
                    containers = driver.find_elements(By.XPATH, '//div[@class="ipc-metadata-list-summary-item__tc"]')
                    if len(containers) < expected_total:
                        print(f"❌ IMDb KHÔNG LOAD ĐỦ SAU KHI CHỜ")
                        print(f"Đang có {len(containers)}, cần {expected_total}")
                        break
                movie_links = [c.find_element(By.TAG_NAME, "a").get_attribute("href") for c in containers]
                
                print(f"Tổng số phim đã tải: {len(containers)}")
                
                # Chỉ lấy phần mới được load thêm
                start_index = 50*page
                new_containers = containers[start_index:]
                new_links = movie_links[start_index:]
        
                print(f"Số phim mới cần xử lí : {len(new_containers)}")
        
                # Crawl từng phim mới
                for i, container in enumerate(new_containers):
                    try:
                        driver.title   # nếu Edge bị tắt → lỗi tại đây
                    except:
                        print(f"⛔ Trình duyệt Edge đã bị tắt khi đang xử lý phim {global_idx}. Dừng crawl!")
                        return data
                    global_idx = start_index + i + 1
                    print(f"\nĐang xử lí phim {global_idx}...")
        
                    row = row_template.copy()
        
                    # Rank + Title
                    try:
                        row['rank'], row['title'] = container.find_element(By.CLASS_NAME, 'ipc-title__text').text.split('.', 1)
                        row['rank'] = row['rank'].strip()
                        row['title'] = row['title'].strip()
                        print("Rank:", row['rank'])
                    except:
                        pass
        
                    # Metadata block (year, runtime, mpa)
                    movie_metadata_container = None
                    try:
                        movie_metadata_container = container.find_element(By.XPATH, value='.//div[@class="sc-b4f120f6-6 kprlzj dli-title-metadata"]')
                        try:
                            metadata = movie_metadata_container.find_elements(By.XPATH, value='./span[@class="sc-b4f120f6-7 hoOxkw dli-title-metadata-item"]')
 
                            for item in metadata:
                                text = item.text.strip()
                                
                                # Kiểm tra năm phát hành (4 chữ số)
                                if text.isdigit() and len(text) == 4:
                                    row['release_year'] = text
                                    print("release_year:", row['release_year'])
                                # Kiểm tra thời lượng (chứa "h" hoặc "m")
                                elif 'h' in text or 'm' in text:
                                    row['run_time'] = text
                                    print("run_time:", row['run_time'])
                                # Kiểm tra MPA rating (các giá trị phổ biến)
                                elif text in ['G', 'PG', 'PG-13', 'R', 'NC-17', 'TV-MA', 'TV-14', 'TV-PG', 'TV-G', 'Not Rated', 'Unrated', 'Approved']:
                                    row['mpa'] = text
                                    print("mpa:", row['mpa'])
                                else:
                                    if len(text) < 15:
                                        row['mpa'] = text                                        
                        except:
                            pass
                    except:
                        pass   
                    try:
                        row['metascore'] = movie_metadata_container.find_element(By.XPATH, './/span[contains(@class,"metacritic-score-box")]').text
                        print("metascore:", row['metascore'])
                    except:
                        pass       
                    try:
                        row['rating'] = container.find_element(By.XPATH, value='.//span[@class="ipc-rating-star--rating"]').text
                    except:
                        pass      
                    try:
                        row['vote_count'] = container.find_element(By.XPATH, value='.//span[@class="ipc-rating-star--voteCount"]').text
                    except:
                        pass
            
                    # Crawl detail page
                    detail_url = new_links[i]
                    try:
                        genres, budget = get_movie_details(driver, detail_url)
                        row['genres'] = genres
                        row['budget'] = budget
                    except:
                        pass
        
                    print("Genres:", row['genres'])
                    print("Budget:", row['budget'])
                    print(" ✓ DONE")
        
                    data.append(row)
        
                # Load more
                try:
                    load_more_btn = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, load_more_xpath)))
                    driver.execute_script("arguments[0].scrollIntoView(true);", load_more_btn)
                    time.sleep(1)
                    driver.execute_script("arguments[0].click();", load_more_btn)
                    time.sleep(3)
                except:
                    print("No more movies to load.")
                    break
    except Exception as e:
        print(f"⚠️ Lỗi ngoài dự kiến: {e}")
        return data
    
    finally:
        try:
            driver.quit()
        except:
            pass
    return data


## 3. Main

In [23]:
data = crawl_imdb_popular_movies("C:/Program Files (x86)/Microsoft/Edge/Application/msedgedriver.exe")



=== PAGE 1 ===
Tổng số phim đã tải: 50
Số phim mới cần xử lí : 50

Đang xử lí phim 1...
Rank: 1
release_year: 2025
run_time: 1h 48m
mpa: PG
metascore: 73
Genres: Animation, Action, Adventure, Comedy, Crime, Family, Mystery
Budget: $150,000,000 (estimated)
 ✓ DONE

Đang xử lí phim 2...
Rank: 2
release_year: 2025
run_time: 2h 29m
mpa: R
metascore: 78
Genres: nan
Budget: nan
 ✓ DONE
⛔ Trình duyệt Edge đã bị tắt khi đang xử lý phim 2. Dừng crawl!


In [24]:
print(data[0])
print(data[-1])
print(len(data))

{'rank': '1', 'title': 'Zootopia 2', 'genres': 'Animation, Action, Adventure, Comedy, Crime, Family, Mystery', 'budget': '$150,000,000 (estimated)', 'release_year': '2025', 'run_time': '1h 48m', 'mpa': 'PG', 'metascore': '73', 'vote_count': ' (17K)', 'rating': '7.7'}
{'rank': '2', 'title': 'Frankenstein', 'genres': nan, 'budget': nan, 'release_year': '2025', 'run_time': '2h 29m', 'mpa': 'R', 'metascore': '78', 'vote_count': ' (170K)', 'rating': '7.5'}
2


In [ ]:
data = pd.DataFrame(data)
data.to_csv('data/cleaned/IMDB_movies.csv', index=False)
